# Tiny Yolo - Keras - Tensorflow backend

YOLO paper: Redmon et al., 2016 (https://arxiv.org/abs/1506.02640)

### Imports

In [1]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, Input, MaxPooling2D, BatchNormalization, Reshape, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

from preprocessing import parse_annotation, BatchGenerator
from yolo_utils import preprocess_image, decode_netout

import numpy as np
import zipfile
import os
import wget

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "" 

%matplotlib inline

c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Constants

In [39]:
with open('tiny-yolo-voc-labels.txt', 'r') as f:
    LABELS = [label.rstrip('\n') for label in f.readlines()]

ANCHORS = [1.08,1.19, 3.42,4.41, 6.63,11.38, 9.42,5.11, 16.62,10.52]

IMAGE_H =  416
IMAGE_W =  416
CHANNELS = 3

GRID_H,  GRID_W  = 13 , 13
BOX = 5
CLASS = len(LABELS)

TRUE_BOX_BUFFER  = 50
BATCH_SIZE       = 64

EPSILON = 1e-8

CLASS_WEIGHTS = np.ones(CLASS, dtype='float32')

OBJECT_SCALE     = 5.0
COORD_SCALE      = 1.0
WARM_UP_BATCHES  = 0

DOWNLOAD_DATA = False

### Model

In [16]:
input_image = Input(shape=(IMAGE_H, IMAGE_W, CHANNELS))
true_boxes  = Input(shape=(1, 1, 1, TRUE_BOX_BUFFER , 4))

'''
https://keras.io/layers/convolutional/

Conv2D(filters, kernel_size, strides=(1, 1), padding='valid',      \
     data_format=None, dilation_rate=(1, 1), activation=None,      \
     use_bias=True, kernel_initializer='glorot_uniform',           \
     bias_initializer='zeros', kernel_regularizer=None,            \
     bias_regularizer=None, activity_regularizer=None,             \
     kernel_constraint=None, bias_constraint=None)
     
https://keras.io/layers/normalization/#batchnormalization

BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,          \
    center=True, scale=True, beta_initializer='zeros',             \
    gamma_initializer='ones', moving_mean_initializer='zeros',     \
    moving_variance_initializer='ones', beta_regularizer=None,     \
    gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

https://keras.io/layers/advanced-activations/#leakyrelu

LeakyReLU(alpha=0.3) 
It allows a small gradient when the unit is not active: f(x) = alpha * x for x < 0, f(x) = x for x >= 0

https://keras.io/layers/pooling/#maxpooling2d

MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)

https://keras.io/layers/core/    -- Reshape

Reshape(target_shape)
'''
# Layer 1
X = Conv2D(filters=16, kernel_size=(3,3), padding='same', use_bias=False, name='conv_1')(input_image)
X = BatchNormalization(name='norm_1')(X)
X = LeakyReLU(alpha=0.1)(X)
X = MaxPooling2D()(X)

# Layer 2
X = Conv2D(filters=32, kernel_size=(3,3), padding='same', use_bias=False, name='conv_2')(X)
X = BatchNormalization(name='norm_2')(X)
X = LeakyReLU(alpha=0.1)(X)
X = MaxPooling2D()(X)

# Layer 3
X = Conv2D(filters=64, kernel_size=(3,3), padding='same', use_bias=False, name='conv_3')(X)
X = BatchNormalization(name='norm_3')(X)
X = LeakyReLU(alpha=0.1)(X)
X = MaxPooling2D()(X)

# Layer 4
X = Conv2D(filters=128, kernel_size=(3,3), padding='same', use_bias=False, name='conv_4')(X)
X = BatchNormalization(name='norm_4')(X)
X = LeakyReLU(alpha=0.1)(X)
X = MaxPooling2D()(X)

# Layer 5
X = Conv2D(filters=256, kernel_size=(3,3), padding='same', use_bias=False, name='conv_5')(X)
X = BatchNormalization(name='norm_5')(X)
X = LeakyReLU(alpha=0.1)(X)
X = MaxPooling2D()(X)

# Layer 6
X = Conv2D(filters=512, kernel_size=(3,3), padding='same', use_bias=False, name='conv_6')(X)
X = BatchNormalization(name='norm_6')(X)
X = LeakyReLU(alpha=0.1)(X)
X = MaxPooling2D(strides=(1,1), padding='same')(X)

# Layer 7
X = Conv2D(filters=1024, kernel_size=(3,3), padding='same', use_bias=False, name='conv_7')(X)
X = BatchNormalization(name='norm_7')(X)
X = LeakyReLU(alpha=0.1)(X)

# Layer 8
X = Conv2D(filters=1024, kernel_size=(3,3), padding='same', use_bias=False, name='conv_8')(X)
X = BatchNormalization(name='norm_8')(X)
X = LeakyReLU(alpha=0.1)(X)

# Layer 9
# BOX=5, CLASS=20, GRID_H=13, GRID_W=13
X = Conv2D(BOX * (4 + 1 + CLASS), kernel_size=(1, 1), kernel_initializer='he_normal')(X)
Y = Reshape((GRID_H, GRID_W, BOX, 4 + 1 + CLASS))(X)       # X = [None, 13, 13, 125], Y = [None, 13, 13, 5, 25]

# small hack to allow true_boxes to be registered when Keras build the model 
# for more information: https://github.com/fchollet/keras/issues/2790
#Y = Lambda(lambda args: args[0])([Y, true_boxes])

# Create model
model = Model([input_image, true_boxes], Y)

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 416, 416, 3)       0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 416, 416, 16)      432       
_________________________________________________________________
norm_1 (BatchNormalization)  (None, 416, 416, 16)      64        
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 416, 416, 16)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 208, 208, 16)      0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 208, 208, 32)      4608      
_________________________________________________________________
norm_2 (BatchNormalization)  (None, 208, 208, 32)      128       
__________

# Update network with pre-trained weights (if any)

In [18]:
if(os.path.isdir("weights")):
    if(os.path.exists("weights/yolov2-tiny-voc.weights")):
        print("file already exists")
    else:
        os.chdir("weights/")
        url = 'https://pjreddie.com/media/files/yolov2-tiny-voc.weights'
        wget.download(url)
        os.chdir("../")
else:
    os.makedirs("weights")
    os.chdir("weights/")
    url = 'https://pjreddie.com/media/files/yolov2-tiny-voc.weights'
    wget.download(url)
    os.chdir("../")

file already exists


In [19]:
class weight_reader:
    def __init__(self, weight_file):
        self.offset = 4
        self.all_weights = np.fromfile(weight_file, dtype='float32')
        
    def read_bytes(self, size):
        read = self.all_weights[self.offset : self.offset + size]
        self.offset = self.offset + size
        return read

In [20]:
if(os.path.exists('weights/yolov2-tiny-voc.weights')):
    wr = weight_reader('weights/yolov2-tiny-voc.weights')
    conv_ = 8
    
    for i in range(1, conv_ + 1):
        conv_layer = model.get_layer('conv_' + str(i))
    
        if i < conv_:
            norm_layer = model.get_layer('norm_' + str(i))
            size = np.prod(norm_layer.get_weights()[0].shape)
        
            beta  = wr.read_bytes(size)
            gamma = wr.read_bytes(size)
            mean  = wr.read_bytes(size)
            var   = wr.read_bytes(size)
        
            weights = norm_layer.set_weights([gamma, beta, mean, var])
        
        if len(conv_layer.get_weights()) > 1:
            bias   = wr.read_bytes(np.prod(conv_layer.get_weights()[1].shape))
            kernel = wr.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
            kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
            kernel = kernel.transpose([2,3,1,0])
            conv_layer.set_weights([kernel, bias])
        else:
            kernel = wr.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
            kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
            kernel = kernel.transpose([2,3,1,0])
            conv_layer.set_weights([kernel])
        
    print('Model loaded with pre-trained weights.')
else:
    print('Weights file doesn\'t exists.')

Model loaded with pre-trained weights.


In [21]:
if(os.path.exists('CheckPoint/weights.10-1.57.hdf5')):
    from keras.models import load_model
    model.load_weights('CheckPoint/weights.10-1.57.hdf5')
    print('Model loaded with pre-trained weights.')

Model loaded with pre-trained weights.


# Download Training and Testing data

In [9]:
if (DOWNLOAD_DATA == True):
    # Save current directory path
    curr_dir = os.getcwd()

    # Training Data
    if(os.path.isdir("data") and os.path.exists("data/train2014.zip")):
        if(os.path.isdir("data/Ext/images/train2014") and len(os.listdir('data/Ext/images/train2014')) != 0):
            print('Training files available on disk.')
        else:
            with zipfile.ZipFile("data/train2014.zip", 'r') as zip_ref:
                zip_ref.extractall("data/Ext/images/")
    else:
        if(os.path.isdir("data")):
            os.chdir("data/")
        else:
            os.makedirs("data/Ext/images")
            os.chdir("data/")
        url = 'http://images.cocodataset.org/zips/train2014.zip'
        filename = wget.download(url)
        with zipfile.ZipFile("train2014.zip", 'r') as zip_ref:
            zip_ref.extractall("./Ext/images")

    os.chdir(curr_dir)

    # Training Data Annotation
    if(os.path.isdir("data") and os.path.exists("data/annotations_trainval2014.zip")):
        if(os.path.isdir("data/Ext/annotations") and len(os.listdir('data/Ext/annotations')) != 0):
            print('Training annotation files available on disk.')
        else:
            with zipfile.ZipFile("data/annotations_trainval2014.zip", 'r') as zip_ref:
                zip_ref.extractall("data/Ext/")
    else:
        if(os.path.isdir("data")):
            os.chdir("data/")
        else:
            os.makedirs("data")
            os.chdir("data/")
        url = 'http://images.cocodataset.org/annotations/annotations_trainval2014.zip'
        filename = wget.download(url)
        with zipfile.ZipFile("annotations_trainval2014.zip", 'r') as zip_ref:
            zip_ref.extractall("./Ext/")

    os.chdir(curr_dir)

    # Validation/Testing Data Annotation
    if(os.path.isdir("data") and os.path.exists("data/val2014.zip")):
        if(os.path.isdir("data/Ext/images/val2014") and len(os.listdir('data/Ext/images/val2014')) != 0):
            print('Validation / Testing files available on disk.')
        else:
            with zipfile.ZipFile("data/val2014.zip", 'r') as zip_ref:
                zip_ref.extractall("data/Ext/images/")
    else:
        if(os.path.isdir("data")):
            os.chdir("data/")
        else:
            os.makedirs("data")
            os.chdir("data/")
        url = 'http://images.cocodataset.org/zips/val2014.zip'
        filename = wget.download(url)
        with zipfile.ZipFile("val2014.zip", 'r') as zip_ref:
            zip_ref.extractall("./Ext/images/")

    os.chdir(curr_dir)

# Convert data from COCO format to VOC

In [10]:
if (DOWNLOAD_DATA == True):
    os.makedirs("data/Ext/images/train2014ann")
    !python coco2pascal.py create_annotations data/Ext train data/Ext/images/train2014ann

In [11]:
if (DOWNLOAD_DATA == True):
    os.makedirs("data/Ext/images/val2014ann")
    !python coco2pascal.py create_annotations data/Ext val data/Ext/images/val2014ann

In [29]:
training_data   = "data/Ext/images/train2014/"
training_data_ann = "data/Ext/images/train2014ann/"
validation_data = "data/Ext/images/val2014/"
validation_data_ann = "data/Ext/images/val2014ann/"

# Parse annotations to generator training and validation generator

In [47]:
gen_config = {
    'IMAGE_H'          : IMAGE_H,
    'IMAGE_W'          : IMAGE_W,
    'GRID_H'           : GRID_H,
    'GRID_W'           : GRID_W,
    'BOX'              : BOX,
    'LABELS'           : LABELS,
    'CLASS'            : len(LABELS),
    'ANCHORS'          : ANCHORS,
    'BATCH_SIZE'       : (BATCH_SIZE * 10),
    'TRUE_BOX_BUFFER'  : TRUE_BOX_BUFFER,
}

In [31]:
def normalize(image):
    return image / 255

In [12]:
train_imgs, seen_train_labels = parse_annotation(training_data_ann, \
                                                 training_data,     \
                                                 labels=LABELS)

train_batch = BatchGenerator(train_imgs, gen_config, norm=normalize)


valid_imgs, seen_valid_labels = parse_annotation(validation_data_ann, \
                                                 validation_data,     \
                                                 labels=LABELS)

valid_batch = BatchGenerator(valid_imgs, gen_config, norm=normalize, jitter=False)

In [30]:
train_imgs, seen_train_labels = parse_annotation(training_data_ann, \
                                                 training_data,     \
                                                 labels=LABELS)



NameError: name 'normalize' is not defined

In [48]:
train_batch = BatchGenerator(train_imgs, gen_config, norm=normalize)

In [49]:
train = 0
for [X1, X2], Y in train_batch:
    np.savez('data/Ext/images/train2014npz/np-{0:0=5d}.npz'.format(train), X1=X1, X2=X2, Y=Y)
    train += 1
    break

In [ ]:
valid = 0
for [X1, X2], Y in valid_batch:
    np.savez('data/Ext/images/valid2014npz/np-{0:0=5d}.npz'.format(train), X1=X1, Y=Y)
    train += 1

In [46]:
for i in range(1):
    X1 = []
    X2 = []
    Y  = []
    file = 'data/Ext/images/train2014npz/np-{0:0=5d}.npz'.format(i)
    data = np.load(file)
    X1.append(data['X1'])
    X2.append(data['X2'])
    Y.append(data['Y'])
    print(len(X1[0]), len(X2[0]), len(Y[0]))
    model.fit([X1[0], X2[0]], Y[0], batch_size=64, epochs=10, verbose=2)

480 480 480
Epoch 1/10


InternalError: cuDNN Backward Filter function launch failure : input shape([64,1024,13,13]) filter shape([3,3,1024,1024])
	 [[{{node training_2/Adam/gradients/conv_8_1/convolution_grad/Conv2DBackpropFilter}} = Conv2DBackpropFilter[T=DT_FLOAT, _class=["loc:@training_2/Adam/gradients/conv_8_1/convolution_grad/Conv2DBackpropInput"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](leaky_re_lu_15/LeakyRelu, ConstantFolding/training_2/Adam/gradients/conv_8_1/convolution_grad/ShapeN-matshapes-1, training_2/Adam/gradients/AddN_7)]]

Caused by op 'training_2/Adam/gradients/conv_8_1/convolution_grad/Conv2DBackpropFilter', defined at:
  File "c:\programdata\anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\programdata\anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\program

# Loss Function

## 1 - Classification Loss

if an object is detected, the classification loss at each cell is the squared error of the class conditional probabilities for each loss:

<img src="images/ClassificationLoss.png" style="width:500px;height:250;">
<caption><center> <u> **Figure 1** </u>: **Classification Loss**<br> </center></caption>

In [40]:
def classification_loss(y_true, y_pred):

    mask_shape = tf.shape(y_true)[:4]
    
    class_mask = tf.zeros(mask_shape)
    
    # true classification box
    true_box_class = tf.argmax(y_true[...,5:], -1)
    
    # classification mask
    class_mask = y_true[...,4] * tf.gather(CLASS_WEIGHTS, true_box_class)

    # number of classification boxes
    nb_class_box = tf.reduce_sum(tf.to_float(class_mask > 0.0))
    
    # predicted box
    pred_box_class = y_pred[...,5:]

    loss_class = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=true_box_class, logits=pred_box_class)
    
    loss_class = tf.reduce_sum(loss_class * class_mask) / (nb_class_box + EPSILON)
    
    loss_class = tf.Print(loss_class, [loss_class], message='Loss Class \t', summarize=1000)
    
    return loss_class

## 2 - Localization Loss

The localization loss measures the errors in the predicted boundary box locations and sizes. We only count the box responsible for detecting the object.

<img src="images/LocalizationLoss.png" style="width:500px;height:250;">
<caption><center> <u> **Figure 2** </u>: **Localization Loss**<br> </center></caption>

We do not want to weight absolute errors in large boxes and small boxes equally. i.e. a 2-pixel error in a large box is the same for a small box. To partially address this, YOLO predicts the square root of the bounding box width and height instead of the width and height. In addition, to put more emphasis on the boundary box accuracy, we multiply the loss by λcoord (default: 5).

In [41]:
def localization_loss(y_true, y_pred):
    mask_shape = tf.shape(y_true)[:4]
    
    cell_x = tf.to_float(tf.reshape(tf.tile(tf.range(GRID_W), [GRID_H]), (1, GRID_H, GRID_W, 1, 1)))
    cell_y = tf.transpose(cell_x, (0,2,1,3,4))

    cell_grid = tf.tile(tf.concat([cell_x,cell_y], -1), [BATCH_SIZE, 1, 1, 5, 1])
    
    coord_mask = tf.zeros(mask_shape)
    
    seen = tf.Variable(0.)
    
    """
    Adjust prediction
    """
    ### adjust x and y      
    pred_box_xy = tf.sigmoid(y_pred[..., :2]) + cell_grid
    
    ### adjust w and h
    pred_box_wh = tf.exp(y_pred[..., 2:4]) * np.reshape(ANCHORS, [1,1,1,BOX,2])
    
    """
    Adjust ground truth
    """
    ### adjust x and y
    true_box_xy = y_true[..., 0:2] # relative position to the containing cell
    
    ### adjust w and h
    true_box_wh = y_true[..., 2:4] # number of cells accross, horizontally and vertically   
    
    """
    Determine the masks
    """
    ### coordinate mask: simply the position of the ground truth boxes (the predictors)
    coord_mask = tf.expand_dims(y_true[..., 4], axis=-1)
    
    no_boxes_mask = tf.to_float(coord_mask < COORD_SCALE/2.)
    seen = tf.assign_add(seen, 1.)
    
    true_box_xy, true_box_wh, coord_mask = tf.cond(tf.less(seen, WARM_UP_BATCHES), 
                          lambda: [true_box_xy + (0.5 + cell_grid) * no_boxes_mask, 
                                   true_box_wh + tf.ones_like(true_box_wh) * np.reshape(ANCHORS, [1,1,1,BOX,2]) * no_boxes_mask, 
                                   tf.ones_like(coord_mask)],
                          lambda: [true_box_xy, 
                                   true_box_wh,
                                   coord_mask])

    nb_coord_box = tf.reduce_sum(tf.to_float(coord_mask > 0.0))
    
    loss_xy    = tf.reduce_sum(tf.square(true_box_xy-pred_box_xy)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_wh    = tf.reduce_sum(tf.square(true_box_wh-pred_box_wh)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    
    Localization_Loss = loss_xy + loss_wh
    
    # Localization_Loss = tf.Print(Localization_Loss, [loss_xy], message='Loss XY \t', summarize=1000)
    # Localization_Loss = tf.Print(Localization_Loss, [loss_wh], message='Loss WH \t', summarize=1000)
    
    return Localization_Loss

## 3 - Confidence Loss

If an object is detected in the box, the confidence loss (measuring the objectness of the box) is:

<img src="images/ConfidenceLoss1.png" style="width:500px;height:250;">
<caption><center> <u> **Figure 3** </u>: **Confidence Loss**<br> </center></caption>

If an object is not detected in the box, the confidence loss is:

<img src="images/ConfidenceLoss2.png" style="width:500px;height:250;">
<caption><center> <u> **Figure 4** </u>: **Confidence Loss**<br> </center></caption>

Most boxes do not contain any objects. This causes a class imbalance problem, i.e. we train the model to detect background more frequently than detecting objects. To remedy this, we weight this loss down by a factor λnoobj (default: 0.5).

In [42]:
def confidence_loss(y_true, y_pred):
    
    mask_shape = tf.shape(y_true)[:4]
    
    cell_x = tf.to_float(tf.reshape(tf.tile(tf.range(GRID_W), [GRID_H]), (1, GRID_H, GRID_W, 1, 1)))
    cell_y = tf.transpose(cell_x, (0,2,1,3,4))

    cell_grid = tf.tile(tf.concat([cell_x,cell_y], -1), [BATCH_SIZE, 1, 1, 5, 1])
    
    conf_mask  = tf.zeros(mask_shape)
    
    """
    Adjust prediction
    """
    ### adjust x and y      
    pred_box_xy = tf.sigmoid(y_pred[..., :2]) + cell_grid
    
    ### adjust w and h
    pred_box_wh = tf.exp(y_pred[..., 2:4]) * np.reshape(ANCHORS, [1,1,1,BOX,2])
    
    ### adjust confidence
    pred_box_conf = tf.sigmoid(y_pred[..., 4])
    
    """
    Adjust ground truth
    """
    ### adjust x and y
    true_box_xy = y_true[..., 0:2] # relative position to the containing cell
    
    ### adjust w and h
    true_box_wh = y_true[..., 2:4] # number of cells accross, horizontally and vertically
    
    ### adjust confidence
    true_wh_half = true_box_wh / 2.
    true_mins    = true_box_xy - true_wh_half
    true_maxes   = true_box_xy + true_wh_half
    
    pred_wh_half = pred_box_wh / 2.
    pred_mins    = pred_box_xy - pred_wh_half
    pred_maxes   = pred_box_xy + pred_wh_half       
    
    intersect_mins  = tf.maximum(pred_mins,  true_mins)
    intersect_maxes = tf.minimum(pred_maxes, true_maxes)
    intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]
    
    true_areas = true_box_wh[..., 0] * true_box_wh[..., 1]
    pred_areas = pred_box_wh[..., 0] * pred_box_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores  = tf.truediv(intersect_areas, union_areas)
    
    true_box_conf = iou_scores * y_true[..., 4]
    
    ### confidence mask: penelize predictors + penalize boxes with low IOU
    # penalize the confidence of the boxes, which have IOU with some ground truth box < 0.6
    true_xy = true_boxes[..., 0:2]
    true_wh = true_boxes[..., 2:4]
    
    true_wh_half = true_wh / 2.
    true_mins    = true_xy - true_wh_half
    true_maxes   = true_xy + true_wh_half
    
    pred_xy = tf.expand_dims(pred_box_xy, 4)
    pred_wh = tf.expand_dims(pred_box_wh, 4)
    
    pred_wh_half = pred_wh / 2.
    pred_mins    = pred_xy - pred_wh_half
    pred_maxes   = pred_xy + pred_wh_half    
    
    intersect_mins  = tf.maximum(pred_mins,  true_mins)
    intersect_maxes = tf.minimum(pred_maxes, true_maxes)
    intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]
    
    true_areas = true_wh[..., 0] * true_wh[..., 1]
    pred_areas = pred_wh[..., 0] * pred_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores  = tf.truediv(intersect_areas, union_areas)

    best_ious = tf.reduce_max(iou_scores, axis=4)
    conf_mask = conf_mask + tf.to_float(best_ious < 0.6) * (1 - y_true[..., 4])
    
    # penalize the confidence of the boxes, which are reponsible for corresponding ground truth box
    conf_mask = conf_mask + y_true[..., 4] * OBJECT_SCALE

    nb_conf_box  = tf.reduce_sum(tf.to_float(conf_mask  > 0.0))

    loss_conf  = tf.reduce_sum(tf.square(true_box_conf - pred_box_conf) * conf_mask)  / (nb_conf_box  + EPSILON) / 2.
    
    # loss_conf = tf.Print(loss_conf, [loss_conf], message='Loss Conf \t', summarize=1000)
    
    return loss_conf

## Total Loss

The final loss adds localization, confidence and classification losses together.

<img src="images/TotalLoss.png" style="width:500px;height:250;">
<caption><center> <u> **Figure 5** </u>: **Total Loss**<br> </center></caption>

In [43]:
def total_loss(y_true, y_pred):
    loss = classification_loss(y_true, y_pred) + localization_loss(y_true, y_pred) + confidence_loss(y_true, y_pred)
    loss = tf.Print(loss, [loss], message='Total Loss \t', summarize=1000)
    return loss

# Optimizer (Adam)

In [44]:
'''
https://keras.io/optimizers/

Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
'''

optimizer = Adam(lr=0.5e-4, epsilon=EPSILON)

# Configure the model for training

In [45]:
'''
https://keras.io/models/sequential/

compile(optimizer, loss=None, metrics=None, loss_weights=None, sample_weight_mode=None, \
                                                        weighted_metrics=None, target_tensors=None)
'''
model.compile(loss=total_loss, optimizer=optimizer)

# Callback functions

In [44]:
'''
https://keras.io/callbacks/

EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', \
                                                  baseline=None, restore_best_weights=False)

ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False,  \
                                              save_weights_only=False, mode='auto', period=1)
'''
EarlyStop = EarlyStopping(min_delta=0.001, patience=5, mode='min', verbose=1)

if(os.path.isdir("CheckPoint")):
    print('Check point directory exists.')
else:
    os.makedirs("CheckPoint")

MakeCheckPoint = ModelCheckpoint("CheckPoint/weights.{epoch:02d}-{val_loss:.2f}.hdf5", \
                                 verbose=1, save_best_only=True, mode='min')

Check point directory exists.


# Start training

In [ ]:
'''
https://keras.io/models/sequential/

fit_generator(generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, \
                   validation_data=None, validation_steps=None, class_weight=None,  \
                   max_queue_size=10, workers=1, use_multiprocessing=False,         \
                   shuffle=True, initial_epoch=0)
'''

model.fit_generator(
                    generator        = train_batch,
                    steps_per_epoch  = len(train_batch),
                    epochs           = 10,
                    validation_data  = valid_batch,
                    validation_steps = len(valid_batch),
                    callbacks        = [EarlyStop, MakeCheckPoint],
                    max_queue_size   = 10
                   )

Epoch 1/10
 131/1425 [=>............................] - ETA: 40:12 - loss: 1.9482

### Non-max suppression ###

Even after filtering by thresholding over the classes scores, you still end up a lot of overlapping boxes. A second filter for selecting the right boxes is called non-maximum suppression (NMS). 

<img src="images/non-max-suppression.png" style="width:500px;height:400;">
<caption><center> <u> **Figure 5** </u>: In this example, the model has predicted 3 cars, but it's actually 3 predictions of the same car. Running non-max suppression (NMS) will select only the most accurate (highest probabiliy) one of the 3 boxes. <br> </center></caption>


Non-max suppression uses the very important function called **"Intersection over Union"**, or IoU.
<img src="images/iou.png" style="width:500px;height:400;">
<caption><center> <u> **Figure 6** </u>: Definition of "Intersection over Union". <br> </center></caption> 

## Test Tiny YOLO

In [ ]:
def predict(xmodel, image_file):

    # Preprocess your image
    image, image_data = preprocess_image("images/" + image_file, model_image_size = (IMAGE_H, IMAGE_W))


    _array = np.zeros((1,1,1,1,TRUE_BOX_BUFFER,4))

    netout = xmodel.predict([image_data, _array])

    return netout

In [ ]:
def decode_network_output(netout):
    return decode_netout(netout[0], 
                        anchors=ANCHORS, 
                        nb_class=CLASS)

Run the following cell on the "test.jpg" image to verify that your function is correct.

In [ ]:
netout = predict(model, "COCO_val2014_000000385918.jpg")

In [ ]:
boxes = decode_network_output(netout)

In [ ]:
for box in boxes:
    print(LABELS[box.get_label()])

**References**: The ideas presented in this notebook came primarily from the two YOLO papers. The implementation here also took significant inspiration and used many components from Allan Zelener's github repository. The pretrained weights used in this exercise came from the official YOLO website. 
- Joseph Redmon, Santosh Divvala, Ross Girshick, Ali Farhadi - [You Only Look Once: Unified, Real-Time Object Detection](https://arxiv.org/abs/1506.02640) (2015)
- Joseph Redmon, Ali Farhadi - [YOLO9000: Better, Faster, Stronger](https://arxiv.org/abs/1612.08242) (2016)
- Allan Zelener - [YAD2K: Yet Another Darknet 2 Keras](https://github.com/allanzelener/YAD2K)
- The official YOLO website (https://pjreddie.com/darknet/yolo/) 

In [ ]:

train_batch = BatchGenerator(train_imgs, gen_config, norm=normalize)



valid_batch = BatchGenerator(valid_imgs, gen_config, norm=normalize, jitter=False)